In [ ]:
from typing import List

import torch
import torch.nn as nn

from transformers import AutoModelForCausalLM, AutoTokenizer


# можете сменить на mps на макбуке, но лично у меня он криво работает
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Знакомство с Transformers

## Создание модели и предсказание следующего токена
Нужно создать модель через `AutoModelForCausalLM`, создать токенайзер через `AutoTokenizer` и олучить следующий токен через жадную генерацию!

**Внимание** на каких-то из функций далее у вас может кончаться видеопамять из-за хранения активаций. Чтобы этого не происходило рекомендуется все вычисления оборачивать в контекстный менеджер `with torch.no_grad()`

In [ ]:
def move_to_device(inputs, device):
    for k, v in inputs.items():
        inputs[k] = v.to(device)
    return inputs

In [ ]:
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2").to(device)
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")


text = "This is a sample text"

inputs = tokenizer(text, return_tensors="pt")
inputs = move_to_device(inputs, device)

outputs = model(**inputs)
logits = outputs.logits
next_token_idx = logits[0][-1].argmax().item()


next_token = tokenizer.decode([next_token_idx])

assert next_token.strip() == "file"



## Используем Generate

Мы с вами помним про различные виды сэмплинга - top_k, top_p, temperature,frequency penalty.
Отличная новость заключается в том, что нам не нужно все это писать самим! Оно уже включено в [GenerationMixin](https://huggingface.co/docs/transformers/v4.44.2/en/main_classes/text_generation#generation), от которого наследуются модели для генерации текста.

Для генерации есть функция [generate](https://huggingface.co/docs/transformers/v4.44.2/en/main_classes/text_generation#transformers.GenerationMixin.generate)

Ваша задача написать для модели выше генерацию по тексту с:
* Температурой - 0.9
* Top-K - 20
* Repetition Penalty (Frequency Penalty) - 1.2
* максимальное число новых токенов - 10


In [ ]:
text = "This is still a sample text, but"
inputs = tokenizer(text, return_tensors="pt")
inputs = move_to_device(inputs, device)

results = []
for i in range(10):
    gens = model.generate(
        **inputs,
        do_sample=True,
        temperature=0.9,
        top_k=20,
        repetition_penalty=1.2,
        max_new_tokens=10,
        pad_token_id=tokenizer.eos_token_id
    )
    results.append(tokenizer.decode(gens[0, inputs.input_ids.size(1):]))

assert len(set(results)) > 1

## Generate Batched
Теперь давайте жадно сгенерируем текст, но забатчуем несколько сэмплов. До этого мы всегда генерировали по батчу размера 1, поэтому у нас не было паддингов!

Когда появляется несколько текстов разной длины, то появляются и паддинги.

Представим себе ситуцию, что у нас батч из двух элементов длины 2 и 5 (токен -1 будет выступать в качестве паддинга **только для удобства визуализации**).

Тогда

```python
input_ids = [
    [3, 2, -1, -1, -1]
    [5, 6,  7,  1,  2]
]
attention_mask = [
    [1, 1, 0, 0, 0],
    [1, 1, 1, 1, 1]
]
```

Представим, что мы сгенерировали еще один токен, тогда

```python
input_ids = [
    [3, 2, -1, -1, -1, 7]
    [5, 6,  7,  1,  2, 8]
]
attention_mask = [
    [1, 1, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1]
]
```

Получается, что у нас паддинги в маске возникают посередине. Мы не будем заниматься реализацией своего алгоритма генерации здесь, но отметим, что добавление паддинга слева значительно упрощает этот процесс.
Тогда исходная последовательность будет:

```python
input_ids = [
    [-1, -1, -1, 3, 2]
    [ 5,  6,  7, 1, 2]
]
attention_mask = [
    [0, 0, 0, 1, 1],
    [1, 1, 1, 1, 1]
]
```

и после генерации следующего токена

```python
input_ids = [
    [-1, -1, -1, 3, 2, 7]
    [ 5,  6,  7, 1, 2, 8]
]
attention_mask = [
    [0, 0, 0, 1, 1, 1],
    [1, 1, 1, 1, 1, 1]
]
```

В качестве задания давайте соберем батч с левым паддингом и проверим, что жадная генерация (10 токенов) совпадает с генерацией на текстах по отдельности!

Для этого нам придется использовать параметр padding_side в конструкторе токенизатора.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2", padding_side="left")
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
texts = ["This is a sample text", "I'm really tired and this is just about"]
inputs = tokenizer(texts, return_tensors="pt", padding=True)
inputs = move_to_device(inputs, device)

batched_generations: List[str] = []
single_generations: List[str] = []

generations_batched = model.generate(
    **inputs,
    max_new_tokens=10
)

batched_generations = tokenizer.batch_decode(generations_batched[:, inputs.input_ids.size(1):])
for text in texts:
    inputs = tokenizer(text, return_tensors="pt")
    inputs = move_to_device(inputs, device)
    g = model.generate(**inputs, max_new_tokens=10)
    single_generations.append(tokenizer.decode(g[0, inputs.input_ids.size(1):]))

assert len(batched_generations) == 2 and len(single_generations) == 2
for s, b in zip(batched_generations, single_generations):
    assert s == b

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


# KV Cache
При генерации есть опция use_cache - это использование KV cache для генерации.
В рамках этой техники в при генерации в декодере считается только аттеншн последнего токена по всем векторам предыдущих токенов, которые посчитали на предыдущих этапах, а для "старых" (левых) токенов аттеншн не пересчитывается, т.к. "новые" (правые) токены на них не влияют.



В рамках данного задания нужно:
1. Посчитать скорость генерации 100 токенов с и без kv cache, сказать, какая техника и во сколько раз быстрее.
2. Подсчитать скорость генерации 1 токена с и без kv cache, сказать, какая техника быстрее и почему.

Чтобы корректно сравнивать время генерации нужно использовать жадный сэмплинг!

**Ответы на оба вопроса нужно оставить письменно прямо здесь**.

1. Генерация 100 токенов должна быть быстрее, но во сколько раз зависит от процессора или видеокарты.
2. Генерация одного не должна иметь разницы по скорости, т.к. kv cache не имеет смысла на генерации одного токена по заданному контекста.

In [ ]:
import time
import numpy as np

text = """
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vestibulum lorem justo, semper dignissim ipsum vitae, sollicitudin aliquet eros. Duis id ultricies erat. Vivamus commodo auctor massa ut mollis. Maecenas lacinia tempus orci, imperdiet ullamcorper felis accumsan et. Etiam mattis neque diam, at egestas nunc eleifend id. Fusce tristique orci nec sollicitudin elementum. Nullam dui est, feugiat ac pellentesque at, posuere non massa.

Suspendisse accumsan ullamcorper dolor sed dictum. Mauris quis varius felis, quis gravida odio. Vestibulum diam arcu, aliquet convallis congue non, rutrum non turpis. Fusce vel orci ac diam suscipit lacinia. Curabitur maximus orci a dui gravida, accumsan convallis libero ornare. Phasellus dapibus, sapien pulvinar lacinia dictum, massa lacus scelerisque tellus, eu porta dolor eros vitae ex. Maecenas maximus, urna id pharetra dictum, dolor lorem sollicitudin ipsum, sit amet vestibulum orci felis quis leo. Pellentesque vel ligula ut urna eleifend condimentum nec et sem. Integer ligula nunc, rutrum ultricies urna et, congue suscipit lectus.
""".strip()

for use_cache in (True, False):
  times = []
  for _ in range(10):
    start = time.time()
    inputs = tokenizer(text, return_tensors="pt")
    inputs = move_to_device(inputs, device)
    model.generate(**inputs, use_cache=use_cache, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
    times.append(time.time() - start)
  print(f"{'with' if use_cache else 'without'} KV caching: {round(np.mean(times), 3)} +- {round(np.std(times), 3)} seconds")

with KV caching: 0.786 +- 0.016 seconds
without KV caching: 1.138 +- 0.003 seconds


In [ ]:
for use_cache in (True, False):
  times = []
  for _ in range(20):
    start = time.time()
    inputs = tokenizer(text, return_tensors="pt")
    inputs = move_to_device(inputs, device)
    model.generate(**inputs, use_cache=use_cache, max_new_tokens=1, pad_token_id=tokenizer.eos_token_id)
    times.append(time.time() - start)
  print(f"{'with' if use_cache else 'without'} KV caching: {round(np.mean(times), 3)} +- {round(np.std(times), 3)} seconds")

with KV caching: 0.013 +- 0.002 seconds
without KV caching: 0.011 +- 0.0 seconds


# Скоринг, Perplixity

Можно не только генерировать текст. Вспомним, что выдает после lm_head - вектор `[batch_size, seq_len, vocab_size]`, где для каждый вектор `[vocab_size]` это распределение вероятностей по следующему токену!

Опустим размерность batch_size=1 для удобства, seq_len = 4. Пусть у нас есть текст `bos мама мыла раму` (`bos` спецсимвол для начала текста)

Тогда вероятность этого текста расписывается через произведение условных вероятностей:

```
P(bos мама мыла раму) = P(мама | bos) * P(мыла | bos мама) * P(раму| bos мама мыла)
```

Т.е. это вероятность слова при условии его левого контекста.
Зачастую ее обозначают как $P(x_i|x_{<i})$ где $x_i$ - i-е слово, $x_{<i}$ - контекст $[x_1, x_2, x_3, ... x_{i-1}]$
Эти вероятности можно взять из выходного вектора!

Давайте попробуем подсчитать вероятность и perplexity текстов!
perplexity как и вероятность мера того насколько модель "уверена" в тексте, т.е. насколько по оценки ее параметрами данный текст вероятен.

$$Perplexity(X) = exp(-\frac {1} {N} \sum_{i}^{N} log P(x_i | x_{<i}))$$

В этом задании нужно:
1. Посчитать вероятность **text**
2. Посчитать перплексию **text**

Еще одна важная деталь:
работать с вероятностями плохо. Т.к. вероятность представляет собой число от 0 до 1, то при перемножении десятков или даже сотен таких числе теряется точность!
Для этого от произведения вероятностей берут логарифм и получают logprobs - логарифмы вероятностей. Их можно складывать, по свойству логарифма логарифм произведения равен произведению логарифма.

$$ p = p_1 * p_2 * p_3 $$
$$log(p) = log (p_1) + log (p_2) + log (p_3)$$
$$exp(log (p)) = p = exp(log (p_1) + log (p_2) + log (p_3)) = exp (log (p_1 * p_2 * p_3)) = p_1 * p_2 * p_3$$

В pytorch для этого есть `torch.log_softmax`, который считается численно стабильно!

In [ ]:
print(f"Beginning of sentence (BOS) token = `{tokenizer.bos_token}`")
print(f"End of sentence (EOS) token  = `{tokenizer.eos_token}`")
text = "<|endoftext|>I'm so very tired of this<|endoftext|>"

inputs = tokenizer(text, return_tensors="pt")
inputs = move_to_device(inputs, device)
input_ids = inputs.input_ids

with torch.no_grad():
    logits = model(**inputs).logits[0] # seq_len, vocab_size
    logits = logits[:-1] # т.к. для последнего токена ничего не генерируем
    probs = torch.log_softmax(logits, dim=1) # превращаем в вероятноси
    targets = input_ids[0, 1:] # сдвигаем, т.к. 0й токен мы не предсказываем, а предсказывем сразу следующий, т.е. 1й
    text_probs = torch.gather(probs, 1, targets.unsqueeze(1)).squeeze(1)
    text_P = text_probs.sum().exp()
    ppl = (-text_probs.mean()).exp()

print(text_P)
print(ppl)

# должно получиться что-то около 2.1783e-14 для вероятности и около 51 для ppl

Beginning of sentence (BOS) token = `<|endoftext|>`
End of sentence (EOS) token  = `<|endoftext|>`
tensor(2.1782e-14, device='cuda:0')
tensor(51.0197, device='cuda:0')


# Chat-Models

## Формат
Мы уже знаем, что все chat-модели принимают входы в своем особом формате.
Он может быть описан текстом, а может быть заложен в шаблон, который доступен через `tokenizer.apply_chat_template`

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("NousResearch/Meta-Llama-3-8B-Instruct", torch_dtype=torch.half).to(device)

/root/.cache/pypoetry/virtualenvs/luka-generative-lm-ra4rWkzy-py3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/root/.cache/pypoetry/virtualenvs/luka-generative-lm-ra4rWkzy-py3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
text = "hello how are you"
inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)

for i in range(5):
    print(tokenizer.decode(model.generate(**move_to_device(inputs, device), max_new_tokens=20, use_cache=True, do_sample=True, pad_token_id=tokenizer.eos_token_id)[0]))
    print("====" * 3)


hello how are you?
- I'm good, thanks for asking. How about you?
- I'm doing well,
hello how are you?") and the user's response. It then uses a simple if-else statement to determine whether
hello how are you?')
    # print('Hello! How are you?')
    # print('I\'m
hello how are you doing today? I am doing well, thanks for asking. I have been busy with work and other
hello how are you?"). The model is then fine-tuned on a downstream task, such as sentiment analysis or question


Видим, что текст зачастую разламывается. Это потому что формат входных данных сильно отличается от того, что модель видела на обучении. У всех chat-моделей свой формат. Где-то он описан просто словами, где-то он заложен в токенайзер. Мы рассмотрим как раз такой случай - за нас есть удобно написанная функция `apply_chat_template`. Давайте используем ее, чтобы получить префикс для генерации модели

In [ ]:
prefix = tokenizer.apply_chat_template(
    conversation=
    [
        {"role": "system", "content": "You are a helpful assistant, who always helps user"},
        {"role": "user", "content": "How to learn about LLMs?"},
        {"role": "assistant", "content": "You can always attend deepschool!"},
        {"role": "user", "content": "Thank you!"},
    ],
    tokenize=False)
print(prefix)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant, who always helps user<|eot_id|><|start_header_id|>user<|end_header_id|>

How to learn about LLMs?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

You can always attend deepschool!<|eot_id|><|start_header_id|>user<|end_header_id|>

Thank you!<|eot_id|>


In [ ]:
prefix = tokenizer.apply_chat_template(
    conversation=
    [{"role": "user", "content": "hello"},
     {"role": "assistant", "content": "I'm good. How can I help you today"},
     {"role": "user", "content": "I love you"},
    ],
    tokenize=False,
add_generation_prompt=True)

reference = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

hello<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I'm good. How can I help you today<|eot_id|><|start_header_id|>user<|end_header_id|>

I love you<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

assert prefix.strip() == reference.strip()

In [ ]:
inputs = tokenizer(prefix, return_tensors="pt")
print(tokenizer.decode(model.generate(**move_to_device(inputs, device), max_new_tokens=12, use_cache=True, do_sample=False, pad_token_id=tokenizer.eos_token_id)[0]))

/root/.cache/pypoetry/virtualenvs/luka-generative-lm-ra4rWkzy-py3.8/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/.cache/pypoetry/virtualenvs/luka-generative-lm-ra4rWkzy-py3.8/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<|begin_of_text|><|begin_of_text|><|start_header_id|>user<|end_header_id|>

hello<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I'm good. How can I help you today<|eot_id|><|start_header_id|>user<|end_header_id|>

I love you<|eot_id|><|start_header_id|>assistant<|end_header_id|>

That's so sweet! I'm happy to hear that.


## Benchmark

Перед нами датасет MMLU - датасет вопросов и ответов в стиле multiple choice.
* question - вопрос
* choices - варианты ответа
* answer - номер правильного ответа

In [ ]:
from datasets import load_dataset
mmlu = load_dataset("cais/mmlu", "global_facts", split="test")
mmlu[1]

{'question': 'What was GDP per capita in the United States in 1850 when adjusting for inflation and PPP in 2011 prices?',
 'subject': 'global_facts',
 'choices': ['About $300', 'About $3k', 'About $8k', 'About $15k'],
 'answer': 1}

In [ ]:
def calc_acc(p, y):
    assert len(p) == len(y)
    return sum(pi == yi for pi, yi in zip(p, y)) / len(p)

In [ ]:
y_true = [sample["answer"] for sample in mmlu]

In [ ]:
def calculate_log_probs(model, inputs):
    logits = model(**inputs).logits[0] # seq_len, vocab_size
    logits = logits[:-1] # т.к. для последнего токена ничего не генерируем
    probs = torch.log_softmax(logits, dim=1) # превращаем в вероятноси
    targets = input_ids[0, 1:] # сдвигаем, т.к. 0й токен мы не предсказываем, а предсказывем сразу следующий, т.е. 1й
    text_probs = torch.gather(probs, 1, targets.unsqueeze(1)).squeeze(1)
    text_P = text_probs.sum().exp()
    ppl = (-text_probs.mean()).exp()

Считаем вероятности по одному question и choice

In [ ]:
from tqdm import tqdm
import numpy as np

preds_single = []
all_single_logps = []
for i, sample in tqdm(enumerate(mmlu)):
    question = sample["question"]
    sample_logps = []
    for choice in sample["choices"]:
        inputs = move_to_device(tokenizer(question + " " + choice, return_tensors="pt"), device)
        input_ids = inputs.input_ids
        with torch.no_grad():
            logits = model(**inputs).logits[0]
            logits = logits[:-1] # т.к. для последнего токена ничего не генерируем
            logps = torch.log_softmax(logits, dim=1) # превращаем в вероятноси
            targets = input_ids[0, 1:] # сдвигаем, т.к. 0й токен мы не предсказываем, а предсказывем сразу следующий, т.е. 1й
            text_logps = torch.gather(logps, 1, targets.unsqueeze(1)).squeeze(1)
            sample_logps.append(text_logps.sum().item())
    all_single_logps.append(sample_logps)
    preds_single.append(np.argmax(sample_logps))

100it [00:13,  7.48it/s]


In [ ]:
print(calc_acc(preds_single, y_true))

0.49


Считаем все сразу

In [ ]:
all_prompts = []
for sample in tqdm(mmlu):
    question = sample["question"]
    sample_logps = []
    for choice in sample["choices"]:
        all_prompts.append(question + " " + choice)
len(all_prompts)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 17760.43it/s]


400

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
all_logps = []
batch_size = 3
for i in tqdm(range(len(all_prompts) // batch_size + 1)):
    batch = all_prompts[i * batch_size: (i + 1) * batch_size]
    inputs = move_to_device(tokenizer(batch, return_tensors="pt", padding="longest"), device)
    input_ids = inputs.input_ids
    mask = inputs.attention_mask
    with torch.no_grad():
        logits = model(**inputs).logits # batch_size, seq_len, d_model
        logits = logits[:, :-1]
        logps = torch.log_softmax(logits, dim=2)
        targets = input_ids[:, 1:]
        text_logps_vect = torch.gather(logps, 2, targets.unsqueeze(2)).squeeze(2) # batch_size, seq_len - 1
        text_logps = text_logps_vect.masked_fill(~mask[:, 1:].bool(), 0).sum(dim=1)
        all_logps += text_logps.tolist()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 134/134 [00:06<00:00, 19.71it/s]


In [ ]:
preds_batched = torch.FloatTensor(all_logps).view(-1, 4)
preds_batched = preds_batched.argmax(dim=1).tolist()
calc_acc(preds_batched, y_true)

0.49